In [598]:
import glob
import pandas as pd
import re
import nilearn as nil
from nilearn import *
from nilearn import image
from os.path import basename
import numpy as np

get all series

get masks

iterate through series

iterate through masks

produce the vector for this mask/series

for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;
this may be slightly different for Go and Stop trials. For Stop trials it is simply the moment of the tone;
for Go trials it's harder to specify.

for each TRIAL, you will end up with a data table with columns (a) time from trial class reveal to image; (b) separate columns for measurement of each ROI
these can be concatenated across trials, runs, and subjects

then create n_{ROI}*n_{trial classes} graphs of samples, where x is the time from trial class reveal to image, and y is the measurement. plot a lowess curve.

potentially overlay the lowess curves for each trial type so that the response within each ROI can be easily compared.


In [599]:
dropbox_data_dir = '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/'
fmriprep_dir = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/'
nii_raw_path = 'sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii'
mask_location = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/brainmaps/striatum/'

get all series

In [600]:
glob_path = fmriprep_dir + nii_raw_path
print("looking up " + glob_path)
nii_raw_files = glob.glob(glob_path)

looking up /Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii


In [601]:
print(nii_raw_files)

['/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV285/ses-wave1/func/s6_sub-DEV285_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV004/ses-wave1/func/s6_sub-DEV004_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV293/ses-wave1/func/s6_sub-DEV293_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii']


get masks

In [602]:
mask_paths = glob.glob(mask_location+"*.nii.gz")
mask_labels = [re.match(".*/(.*)\.nii\.gz",mp)[1] for mp in mask_paths]

mask_df = pd.DataFrame({
    'mask_label':mask_labels,
    'mask_path': mask_paths
})

In [603]:
roi_data = {'asdf':0}


iterate through series

In [604]:
mydt=pd.DataFrame(index=range(0,6))
mydt['myvar']=['one','two','3','6','4','5']
mydt

,myvar
0,one
1,two
2,3
3,6
4,4
5,5


In [605]:
#mark the number of seconds each scan takes.
TR = 2.0

roi_data = {}

for nii in nii_raw_files:
    series_filename = basename(nii)
    match_data = re.match('.*(DEV\d*)_ses-wave(\d+)*',series_filename)
    dev_name = match_data[1]
    dev_wave = match_data[2]

    #print(basename(nii))
    print(dev_name + ", " + dev_wave)

    active_img = image.load_img(nii)

    #create a template data frame to populate with the ROI data, for this run
    run_len = active_img.shape[3]
    if dev_name not in roi_data.keys():
        roi_data[dev_name]={}
    
    run_df = pd.DataFrame(index=range(0,run_len))
    #run_df = pd.DataFrame({'image_id':range(0,run_len)})
    #run_df['TR_onset']=run_df.image_id*TR


    
    for m_i, m_set in mask_df.iterrows():
        print(m_set['mask_label'])
        print('producing the vector for this mask/series...')
        
        #active_mask = nilearn.masking.compute_brain_mask(m_set['mask_path'])
        mask_raw = nilearn.image.load_img(m_set['mask_path'])
        mask_in_subj_space = nilearn.image.resample_img(mask_raw, target_affine=active_img.affine,target_shape = active_img.slicer[:,:,:,0].shape)
        mask_binarized = nilearn.image.binarize_img(mask_in_subj_space,threshold=50)
        active_img_masked = nilearn.masking.apply_mask(active_img, mask_binarized)
        activity_vector = active_img_masked.mean(axis=1)
        run_df[m_set['mask_label']]=activity_vector
        print("created an activity vector for this mask with the following length: " + str(len(activity_vector)))
        #
        #nil.plotting.plot_img(mask_applied)
        #nilearn.plotting.view_img(active_mask)
    #now we've gone through all the masks for this run, add the run_df to the dict dict
    roi_data[dev_name][dev_wave] = run_df



        

        



DEV285, 1


KeyboardInterrupt: 

In [ ]:
for s in roi_data.keys():
    print(s)
    for wave in roi_data[s].keys():
        print(wave)
        print(roi_data[s][wave].shape)
        print(roi_data[s][wave].columns)

DEV004
1
(207, 6)
Index(['harvardoxford-subcortical_prob_Left Putamen',
       'harvardoxford-subcortical_prob_Right Accumbens',
       'harvardoxford-subcortical_prob_Right Putamen',
       'harvardoxford-subcortical_prob_Left Accumbens',
       'harvardoxford-subcortical_prob_Left Caudate',
       'harvardoxford-subcortical_prob_Right Caudate'],
      dtype='object')
DEV293
1
(212, 6)
Index(['harvardoxford-subcortical_prob_Left Putamen',
       'harvardoxford-subcortical_prob_Right Accumbens',
       'harvardoxford-subcortical_prob_Right Putamen',
       'harvardoxford-subcortical_prob_Left Accumbens',
       'harvardoxford-subcortical_prob_Left Caudate',
       'harvardoxford-subcortical_prob_Right Caudate'],
      dtype='object')


In [ ]:
sst_all_behavioral_data.trial_n

0           1
1           2
2           3
3           4
4           5
         ... 
259835    252
259836    253
259837    254
259838    255
259839    256
Name: trial_n, Length: 259840, dtype: int64

In [ ]:
pd.Series([0.]*240)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
235    0.0
236    0.0
237    0.0
238    0.0
239    0.0
Length: 240, dtype: float64

In [ ]:
sst_all_behavioral_data = pd.read_csv(dropbox_data_dir + "sst_behavioral_data_all.csv")



for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;

In [ ]:
# for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;

# this may be slightly different for Go and Stop trials. For Stop trials it is simply the moment of the tone;
# for Go trials it's harder to specify.

def get_moment_trial_type_revealed(trial_type, ssdr,first_tone):
    trial_type = trial_type.tolist()
    ssdr = ssdr.tolist()
    expected_tone_time = pd.Series([0.]*len(ssdr))
    expected_tone_time[0]=first_tone
    expected_tone_time[1]=first_tone

    for i in range(2,len(expected_tone_time)):
         #these are in increments of 2 because every second event is a cue
         #previous trial was a stop
        if trial_type[i]=="Stop":
            expected_tone_time[i] = ssdr[i]
        else:
            expected_tone_time[i] = expected_tone_time[i-1]
    #print(expected_tone_time)
    return(expected_tone_time)




# but might be the expected time of the onset, which would be the _previous_ stop trial.
#create expected tone table
expected_tone_table = sst_all_behavioral_data.groupby(['subid', 'waveid','runid']).apply(
    lambda sabd_set: get_moment_trial_type_revealed(sabd_set['go_no_go_condition_label'], sabd_set['SSD_recorded'],first_tone=2)
)
expected_tone_table2=expected_tone_table.reset_index()
expected_tone_table2.rename(columns={0:'class_type_reveal'},inplace=True)
expected_tone_table2.loc[:,'trial_n']=expected_tone_table2.level_3+1
expected_tone_table2.drop('level_3',axis=1,inplace=True)

sst_all_behavioral_data = sst_all_behavioral_data.merge(expected_tone_table2,
left_on=['subid','waveid','runid','trial_n'],
right_on =['subid','waveid','runid','trial_n'],
how='left'
    
    )


In [ ]:
sst_all_behavioral_data['class_type_reveal_onset'] = sst_all_behavioral_data.class_type_reveal + sst_all_behavioral_data.onset
sst_all_behavioral_data.loc[sst_all_behavioral_data.go_no_go_condition_label=='Cue','class_type_reveal_onset']=None

Now we need a list of the TRs by their time of appearance; that way we can mark which ones are relevant to the ROIs for each trial.

For now, let's assume that the timing of the TRs matches the timing in the matlab files, but this MUST be checked.

In [ ]:
run_length = 207
rt_time_list = [x*2.0 for x in range(run_length)]

these are going to be generated dynamically based on the run length

In [ ]:
# (is there a way to do this vectorized over ALL the data,
# or is it more sensible to loop through subjects/runs to do it?
# tend to think looping through subjects makes more sense.)
all_subj_df_list = [] 
for s in roi_data.keys():
    print(s)
    for wave in roi_data[s].keys():
        run_length = roi_data[s][wave].shape[0]
        print(s +', ' + wave + ':' + str(run_length))
        rt_time_list = [x*2.0 for x in range(run_length)]
        #now we have a list of the the RTs
        #we have actual data at each data point
        #we need to get the RTs related to each particular trial.
        run_behavioral_data = sst_all_behavioral_data[(sst_all_behavioral_data.subid==s) & (sst_all_behavioral_data.waveid==int(wave))]

        run_trial_df_list = []
        for i, trial in run_behavioral_data.iterrows():
            #print(i)
            t_class_type_reveal_onset = trial['class_type_reveal_onset']
            #print(trial[['trial_n','class_type_reveal_onset']])
            #grab the set of TR times that are relevant based on its
            

            trial_rt_timing_all = pd.DataFrame({'tr':rt_time_list,'offset':[(tr - t_class_type_reveal_onset) for tr in rt_time_list]})
            trs_to_use_for_trial = ((trial_rt_timing_all.offset>-10) & (trial_rt_timing_all.offset<20))
            trial_rt_timing_inscope = trial_rt_timing_all.loc[trs_to_use_for_trial]

            #and we will also want to grab the actual data we already extracted...
            trial_roi_data = pd.merge(trial_rt_timing_inscope,roi_data[s][wave],how='left',
            left_index=True,right_index=True
            )
            trial_roi_data['t_class_type_reveal_onset']=t_class_type_reveal_onset
            trial_roi_data['subid']=s
            trial_roi_data['wave']=wave
            trial_roi_data['trial_n']=trial[['trial_n']][0]
            run_trial_df_list.append(trial_roi_data)
            #cool. now what? 

        run_trial_df = pd.concat(run_trial_df_list)
        run_trial_df.reset_index(drop=True,inplace=True)

        all_subj_df_list.append(run_trial_df)
all_subj_df = pd.concat(all_subj_df_list)

DEV004
DEV004, 1:207
DEV293
DEV293, 1:212


In [ ]:
all_subj_df.groupby(['trial_n','subid','wave']).offset.count()

trial_n  subid   wave
1        DEV004  1       12
         DEV293  1       12
3        DEV004  1       12
         DEV293  1       13
5        DEV004  1       14
                         ..
251      DEV293  1       11
253      DEV004  1        3
         DEV293  1        9
255      DEV004  1        1
         DEV293  1        7
Name: offset, Length: 256, dtype: int64

In [ ]:
all_subj_df.loc[(all_subj_df.trial_n==255) & (all_subj_df.subid=='DEV293')]

,tr,offset,harvardoxford-subcortical_prob_Left Putamen,harvardoxford-subcortical_prob_Right Accumbens,harvardoxford-subcortical_prob_Right Putamen,harvardoxford-subcortical_prob_Left Accumbens,harvardoxford-subcortical_prob_Left Caudate,harvardoxford-subcortical_prob_Right Caudate,t_class_type_reveal_onset,subid,wave,trial_n
1886,410.0,-8.151915,6778.073129,7391.719822,6605.091435,7485.192478,7673.730703,7605.616266,418.151915,DEV293,1,255
1887,412.0,-6.151915,6767.028584,7345.253338,6595.588242,7467.411279,7661.025504,7587.698546,418.151915,DEV293,1,255
1888,414.0,-4.151915,6781.452559,7403.892680,6602.077164,7504.595498,7681.999258,7630.595061,418.151915,DEV293,1,255
1889,416.0,-2.151915,6789.747892,7432.853785,6621.772759,7563.510660,7679.435374,7626.020432,418.151915,DEV293,1,255
1890,418.0,-0.151915,6792.449068,7431.663162,6606.022412,7554.434342,7667.837597,7623.877606,418.151915,DEV293,1,255
1891,420.0,1.848085,6785.438189,7316.650799,6598.487077,7540.463137,7646.919504,7596.633747,418.151915,DEV293,1,255
1892,422.0,3.848085,6785.893177,7332.000185,6624.031078,7494.312927,7663.541265,7611.590154,418.151915,DEV293,1,255


We should have up to 420 or so.

In [ ]:
s='DEV004'
wave=(list(roi_data[s].keys())[0])
run_behavioral_data = sst_all_behavioral_data.loc[
    (sst_all_behavioral_data.subid==s) & (sst_all_behavioral_data.waveid==int(wave)) & (sst_all_behavioral_data.go_no_go_condition_label!='Cue'),:]

#move the below up to where roi_data is first created.
rt_time_list = [x*2.0 for x in range(run_length)]
#roi_data[s][wave].index = pd.Series(rt_time_list,name='tr_time')


In [606]:
sst_all_behavioral_data.loc[sst_all_behavioral_data.subid=='DEV293']

,trial_n,numchunks_reciprocal,go_no_go_condition,arrow_presented,ladder_number,LadderX_SSD_ms,subject_response,ladder_movement,reaction_time,SSD_technical,...,time_course,health_condition,subid,waveid,runid,rundatetime,go_no_go_condition_label,condition,class_type_reveal,class_type_reveal_onset
248832,1,1,1,1,1,250,94,-1,0.425402,0,...,0.00000,1,DEV293,1,1,2022-06-21T22:11:00Z,Stop,FailedStop,2.000000,2.011194
248833,2,1,2,2,0,0,0,0,0.000000,0,...,2.25834,2,DEV293,1,1,2022-06-21T22:11:00Z,Cue,Cue,2.000000,NaN
248834,3,1,0,1,0,0,94,0,0.480118,0,...,2.75834,0,DEV293,1,1,2022-06-21T22:11:00Z,Go,CorrectGo,2.000000,4.763036
248835,4,1,2,2,0,0,0,0,0.000000,0,...,4.76390,2,DEV293,1,1,2022-06-21T22:11:00Z,Cue,Cue,2.000000,NaN
248836,5,1,0,0,0,0,91,0,0.449974,0,...,5.51390,1,DEV293,1,1,2022-06-21T22:11:00Z,Go,CorrectGo,2.000000,7.528667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249083,252,1,2,2,0,0,0,0,0.000000,0,...,412.82572,2,DEV293,1,1,2022-06-21T22:11:00Z,Cue,Cue,0.301459,NaN
249084,253,1,0,0,0,0,91,0,0.519567,0,...,414.57572,1,DEV293,1,1,2022-06-21T22:11:00Z,Go,CorrectGo,0.301459,414.885800
249085,254,1,2,2,0,0,0,0,0.000000,0,...,417.08682,2,DEV293,1,1,2022-06-21T22:11:00Z,Cue,Cue,0.301459,NaN
249086,255,1,1,1,1,300,0,1,0.000000,0,...,417.83682,0,DEV293,1,1,2022-06-21T22:11:00Z,Stop,CorrectStop,0.302115,418.151915


In [ ]:
418/2

209.0

In [ ]:
# tr_relative_to_trial = [roi_data[s][wave].index- t_class_type_reveal_onset][0]
# trs_to_select = (tr_relative_to_trial> -10) & (tr_relative_to_trial<20)
# roi_data[s][wave][trs_to_select]

,harvardoxford-subcortical_prob_Left Putamen,harvardoxford-subcortical_prob_Right Accumbens,harvardoxford-subcortical_prob_Right Putamen,harvardoxford-subcortical_prob_Left Accumbens,harvardoxford-subcortical_prob_Left Caudate,harvardoxford-subcortical_prob_Right Caudate
tr_time,,,,,,
8.0,6461.595717,6590.608791,6059.082021,6989.466081,7021.428720,6846.671920
10.0,6491.308885,6606.488079,6069.866238,6909.105310,7014.371645,6815.043140
12.0,6446.349825,6518.123376,6031.793488,6937.336585,6999.554084,6811.870349
14.0,6412.815328,6534.859846,6020.095737,6924.907452,7003.723867,6804.982511
16.0,6437.908445,6580.183321,6029.871972,6925.096032,6999.080414,6804.419904
18.0,6432.376921,6558.400194,5998.432855,6917.908565,6981.260015,6825.013831
20.0,6439.525207,6539.247544,6031.810552,6919.027187,6979.309252,6812.228790
22.0,6430.698659,6514.367850,6006.724896,6903.165042,6990.359750,6800.746215
24.0,6484.056126,6568.847095,6045.255206,6953.005863,6997.739813,6820.877320


In [ ]:
roi_data['DEV293']['1']

,harvardoxford-subcortical_prob_Left Putamen,harvardoxford-subcortical_prob_Right Accumbens,harvardoxford-subcortical_prob_Right Putamen,harvardoxford-subcortical_prob_Left Accumbens,harvardoxford-subcortical_prob_Left Caudate,harvardoxford-subcortical_prob_Right Caudate
0,6743.436848,7342.807734,6591.830279,7576.720602,7694.697500,7593.772702
1,6724.579021,7290.700730,6586.227727,7544.729154,7654.777559,7553.626823
2,6776.807280,7326.332083,6621.634508,7569.089581,7667.100089,7575.771049
3,6799.255964,7353.909491,6637.887460,7587.977737,7713.509752,7621.046416
4,6832.166333,7451.945129,6659.078952,7581.519871,7749.921891,7653.436099
...,...,...,...,...,...,...
207,6781.452559,7403.892680,6602.077164,7504.595498,7681.999258,7630.595061
208,6789.747892,7432.853785,6621.772759,7563.510660,7679.435374,7626.020432
209,6792.449068,7431.663162,6606.022412,7554.434342,7667.837597,7623.877606
210,6785.438189,7316.650799,6598.487077,7540.463137,7646.919504,7596.633747


0
trial_n                           1
class_type_reveal_onset    2.046817
Name: 0, dtype: object
2
trial_n                           3
class_type_reveal_onset    3.270994
Name: 2, dtype: object
4
trial_n                           5
class_type_reveal_onset    6.657806
Name: 4, dtype: object
6
trial_n                            7
class_type_reveal_onset    12.145926
Name: 6, dtype: object
8
trial_n                            9
class_type_reveal_onset    16.033492
Name: 8, dtype: object
10
trial_n                           11
class_type_reveal_onset    20.905798
Name: 10, dtype: object
12
trial_n                           13
class_type_reveal_onset    23.725689
Name: 12, dtype: object
14
trial_n                           15
class_type_reveal_onset    26.361941
Name: 14, dtype: object
16
trial_n                           17
class_type_reveal_onset    29.315329
Name: 16, dtype: object
18
trial_n                           19
class_type_reveal_onset    31.801615
Name: 18, dtype: object
20
tri

In [ ]:
run_trial_df_list

[      tr     offset  harvardoxford-subcortical_prob_Left Putamen  \
 0    0.0  -2.046817                                  6486.305242   
 1    2.0  -0.046817                                  6482.829658   
 2    4.0   1.953183                                  6506.433352   
 3    6.0   3.953183                                  6488.954113   
 4    8.0   5.953183                                  6461.595717   
 5   10.0   7.953183                                  6491.308885   
 6   12.0   9.953183                                  6446.349825   
 7   14.0  11.953183                                  6412.815328   
 8   16.0  13.953183                                  6437.908445   
 9   18.0  15.953183                                  6432.376921   
 10  20.0  17.953183                                  6439.525207   
 11  22.0  19.953183                                  6430.698659   
 
     harvardoxford-subcortical_prob_Right Accumbens  \
 0                                      6580.7

In [ ]:

# for each TRIAL, you will end up with a data table with columns (a) time from trial class reveal to image; (b) separate columns for measurement of each ROI
# these can be concatenated across trials, runs, and subjects

# then create n_{ROI}*n_{trial classes} graphs of samples, where x is the time from trial class reveal to image, and y is the measurement. plot a lowess curve.

# potentially overlay the lowess curves for each trial type so that the response within each ROI can be easily compared.
